<a href="https://colab.research.google.com/github/Atsumu-Hiranishi/Test/blob/main/8_3hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#do記法
do記法とはモナド専用構文で複数のモナド値を糊付けするもの。<br>


```
foo :: Maybe String
foo = Just 3 >>= (\x ->
      Just "!" >>=(\y ->
      Just (show x ++ y)))
```


このコードをには各行にラムダ式が入っているがdo記法を用いると先ほどのコードは


```
foo :: Maybe String
foo = do
  x <- Just 3
  y <- Just "!"
  Just (show x ++ y)
```


と書くことが出来る。このようにモナド値を連鎖させた式と等価なものを簡潔に表せる記法がdo式である。<br>
do式はlet行を除いてすべてモナド値で構成される。モナドの結果を調べるには`<-`を用いる。また。各モナドが直前のモナドの結果に依存する場合はdo記法ではなく`>>=`を用いた方がよい。<br>

#リストモナド
リスト`[8,9,3]`といった値は複数の計算結果を含んでいるとも、複数の候補値を同時に重ね合わせたような一つの値であるとも解釈できる。このような値を非決定性計算という。


```
ghci> (*) <$> [1,2,3] <*> [10,100,1000]
[10,100,1000,20,200,2000,30,300,3000]
```


このように左辺リストの要素と右辺リストの要素すべての組み合わせの積が答のリストに含まれている。非決定性計算ではたくさんの選択肢に出くわすが、その都度すべてを試す。<br>
この非決定性計算はモナドに焼き直すことが出来る。


```
instance Monad [] where
  return x = [x]
  xs >>= f = concat (map f xs)
  fail _ = []
```


非決定的値を関数に食わせると次のようになる


```
ghci> [3,4,5] >>= \x -> [x,-x]
[3,-3,4,-4,5,-5]
```


`Maybe`に`>>=`を使うと失敗の可能性を考慮しながらモナド値を関数に供給できる。今回はモナドが非決定性計算を処理している。<br>
`Maybe`の時と同様に`>>=`を用いるとリストをいくつでも連結して非決定性を伝播させることが出来る。


```
ghci> [1,2] >>= \n -> [a',b'] >>= \ch -> return (n, ch)
[(1,a')(1,b')(2,a')(2,b')]
```


これをdo記法で書き直すと


```
listOfTuples :: [(Int, Char)]
listOfTuples = do
  n <- [1,2]
  ch <- [a',b']
  return (n, ch)
```


また、これはリスト内包的表記を用いて


```
ghci> [ (n, ch) | n <- [1,2], ch <- [a',b'] ]
[(1,a')(1,b')(2,a')(2,b')]
```


とも表せる。リスト内包的表記はリストモナドの構文糖衣にすぎない。<br>
リスト内包的表記は出力する要素を選別することが出来た。


```
ghci> [ x | x <- [1..500], '7' `elem` show x ]
[7,17,27,37,47]
```


この選別はどのようなリストモナドに翻訳されるのだろうか。
- MonadPlus関数
MonadPlusはモノイドの性質を併せ持つモナドを表す型クラスのこと<br>
定義



```
class Monad m => MonadPlus m where
  mzero :: m a
  mplus :: m a -> m a -> m a
```


`mzero`はMonoid型クラスでいう`mempty`に対応する概念、`mplus`は`mappend`に対応している。<br>
- guard関数
定義


```
guard :: (MonadPlus m) => Bool => m ()
guard True = return ()
guard False = mzero
```


guardは真理値を引数に取り、引数がTrueなら()を成功を表す最小限の文脈に入れる。Falseならば失敗したモナド値を作る。


```
ghci> guard (5 > 2) :: Maybe ()
Just ()
ghci> guard (1 > 2) :: Maybe ()
Nothing
```


リストモナドでは`guard`を使って解の候補を振るい落とすことが出来る。


```
ghci> [1..50] >>= (\x => guard ('7' `elem` show x) >> return x )
[7,17,27,37,47]
```


do記法で表すと


```
sevenOnly ;; [Int]
sevenOnly = do
  x <- [1..50]
  guard ('7' `elem` show x)
  return x
```









